#**Regression with Keras - Tutorial 1**





Tutorial 1 de PyimageSearch de cómo hacer una predicción del precio de la vivienda en Keras usando una regresión básica.

In [1]:
# Primero cargamos librerías y funciones necesarias
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt # para dibujar
%matplotlib inline

#Para trabajar en el colab
from google.colab import drive
drive.mount('/content/drive')

# Nombre de las columnas del DF
cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]

# Carga del DF desde el fichero de texto HousesInfo.txt, poniéndole a las columnas el nombre definido antes
HousesInfo_DF = pd.read_csv('/content/drive/My Drive/BootCamp - BigDataIV - DL/HousesInfo.txt', sep=' ', decimal='.', header=None, names=cols) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# El DF quedaría con 535 filas y 5 columnas :
HousesInfo_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 5 columns):
bedrooms     535 non-null int64
bathrooms    535 non-null float64
area         535 non-null int64
zipcode      535 non-null int64
price        535 non-null int64
dtypes: float64(1), int64(4)
memory usage: 21.0 KB


In [0]:
	# Si hay menos de 25 casas por cada código postal, se elimina.
	
	zipcodes = HousesInfo_DF["zipcode"].value_counts().keys().tolist() # Lista de los CP
	counts = HousesInfo_DF["zipcode"].value_counts().tolist() # Número de veces que aparece cada CP anterior
 
	# Recorremos cada CP y el número de veces que aparece. Si es menos de 25, se elimina los reg con ese CP
	for (zipcode, count) in zip(zipcodes, counts):
		if count < 25:
			idxs = HousesInfo_DF[HousesInfo_DF["zipcode"] == zipcode].index
			HousesInfo_DF.drop(idxs, inplace=True)

In [4]:
HousesInfo_DF.info() #Nos quedamos con 362 filas en total

<class 'pandas.core.frame.DataFrame'>
Int64Index: 362 entries, 30 to 503
Data columns (total 5 columns):
bedrooms     362 non-null int64
bathrooms    362 non-null float64
area         362 non-null int64
zipcode      362 non-null int64
price        362 non-null int64
dtypes: float64(1), int64(4)
memory usage: 17.0 KB


In [5]:
#Separamos en train/test los datos del DF

from sklearn.model_selection import train_test_split

train_DF, test_DF = train_test_split(HousesInfo_DF, test_size=0.2, shuffle=True, random_state=0)
# Muestra por pantalla en número de registros y columnas de cada dataset
print(f'Dimensiones del dataset de training : {train_DF.shape}') 
print(f'Dimensiones del dataset de test     : {test_DF.shape}') 

Dimensiones del dataset de training : (289, 5)
Dimensiones del dataset de test     : (73, 5)


In [0]:
from sklearn.preprocessing import MinMaxScaler

# Nombre de columnas continuas
continuous = ["bedrooms", "bathrooms", "area"]
 
# Escalar cada columna continua en el rango [0,1], en train y test
cs = MinMaxScaler()
trainContinuous = cs.fit_transform(train_DF[continuous])
testContinuous = cs.transform(test_DF[continuous])

In [0]:
from sklearn.preprocessing import LabelBinarizer	
  
# Idem para la columna código postal, usando one-hot, y transformarlo a un array
zipBinarizer = LabelBinarizer().fit(HousesInfo_DF["zipcode"])
trainCategorical = zipBinarizer.transform(train_DF["zipcode"])
testCategorical = zipBinarizer.transform(test_DF["zipcode"])
 
# Unimos categóricas y continuas en un solo array 
trainX = np.hstack([trainCategorical, trainContinuous])
testX = np.hstack([testCategorical, testContinuous])

In [0]:
# Y ahora, escalar el precio para que tenga valores en el rango [0,1]

maxPrice = train_DF["price"].max()
trainY = train_DF["price"] / maxPrice
testY = test_DF["price"] / maxPrice

In [9]:
# Creamos un perceptrón multicapa (MLP), compilar el modelo usando el mean absolute percentage error para
# las pérdidas, buscando minimizar la diferencia entre nuestras predicciones de precios y los precios actuales

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam

model = Sequential()

dim=trainX.shape[1] #La dimensión de entrada será la del número de columnas de train (10)

model.add(Dense(8, input_dim=dim, activation="relu")) # 8 neuronas con función de activación relu
model.add(Dense(4, activation="relu")) # 4 neuronas con función de activación relu
model.add(Dense(1, activation="linear")) # neurona con función de activación lineal

opt = Adam(lr=1e-3, decay=1e-3 / 200) # optimizador algoritmo Adam, learning rate bajo, disminución exp
model.compile(loss="mean_absolute_percentage_error", optimizer=opt) # compila pérdidas con modelo mabspererror

print("[INFO] training model...")
model.fit(trainX, trainY, validation_data=(testX, testY), 
	        epochs=200, batch_size=8)

Using TensorFlow backend.






[INFO] training model...



Train on 289 samples, validate on 73 samples
Epoch 1/200





289/289 [==============================] - 1s 2ms/step - loss: 73.5695 - val_loss: 43.1762
Epoch 2/200
289/289 [==============================] - 0s 261us/step - loss: 48.6998 - val_loss: 33.9878
Epoch 3/200
289/289 [==============================] - 0s 242us/step - loss: 39.6104 - val_loss: 31.1628
Epoch 4/200
289/289 [==============================] - 0s 160us/step - loss: 37.4092 - val_loss: 30.4224
Epoch 5/200
289/289 [==============================] - 0s 161us/step - loss: 35.2360 - val_loss: 29.0705
Epoch 6/200
289/289 [==============================] - 0s 156us/step - loss: 33.3314 - val_loss: 28.5339
Epoch 7/200
289/289 [==============================] - 0s 166us/step - loss: 32.4446 - val_loss: 27.9631
Epoch 8/200
289/289 [==============================] - 0s 164us/step - loss: 31.0933 - val_loss: 26.8890
Epoch 9/200
289/289 [==============================] - 0s 163us/step - loss: 31.

In [0]:
# Como puede verse, nuesta pérdida de entrenamiento es alrededor 20 % y la de validación 20 %

# predicciones sobre el test
preds = model.predict(testX)

# Calcula la diferencia entre los precios predecidos y los actuales 
diff = preds.flatten() - testY # diferencia
percentDiff = (diff / testY) * 100 # porcentaje
absPercentDiff = np.abs(percentDiff) # valor absoluto del porcentaje de la diferencia

# Media y desviación estandard del anterior
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

In [11]:
# Estadísticas del modelo 

import locale  # base de datos regional POSIX

locale.setlocale(locale.LC_ALL, "en_US.UTF-8") # situa en USA la moneda 
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(HousesInfo_DF["price"].mean(), grouping=True), # muestra la media del precio del DF 
	locale.currency(HousesInfo_DF["price"].std(), grouping=True))) # muestra la desv stand del precio del DF
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std)) # lo mismo, pero del precio predicho

[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 23.85%, std: 19.65%


In [0]:
# Este porcentaje de error porcentual absoluto medio de alrededor 20% no muy bueno